<a href="https://colab.research.google.com/github/cristhiansito123/MLDS6project/blob/entrega4/scripts/training/several_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


import neccesary packages

In [2]:
import json
import requests
import os
import shutil
import time
import sys

import numpy as np
import pandas as pd

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
%matplotlib inline

#Carga de datos

In [4]:

metadata_path = '/content/drive/MyDrive/MLDS6project/data.csv'

metadata = pd.read_csv(metadata_path, sep=',')


In [5]:
metadata

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


# Modelo

In [6]:
mapeo = {'CASH_OUT': 1, 'PAYMENT': 2, 'CASH_IN': 3, 'TRANSFER': 4, 'DEBIT': 5}

# Aplicar el mapeo a la columna del DataFrame
metadata['type_numeric'] = metadata['type'].map(mapeo)

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from sklearn.svm import OneClassSVM


In [14]:
feature_names = ['step', 'type_numeric', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
X = metadata[feature_names]
y = metadata['isFraud']

In [15]:
X

,step,type_numeric,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,2,9839.64,170136.00,160296.36,0.00,0.00
1,1,2,1864.28,21249.00,19384.72,0.00,0.00
2,1,4,181.00,181.00,0.00,0.00,0.00
3,1,1,181.00,181.00,0.00,21182.00,0.00
4,1,2,11668.14,41554.00,29885.86,0.00,0.00
...,...,...,...,...,...,...,...
6362615,743,1,339682.13,339682.13,0.00,0.00,339682.13
6362616,743,4,6311409.28,6311409.28,0.00,0.00,0.00
6362617,743,1,6311409.28,6311409.28,0.00,68488.84,6379898.11
6362618,743,4,850002.52,850002.52,0.00,0.00,0.00


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [25]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import balanced_accuracy_score,roc_auc_score,f1_score,average_precision_score,accuracy_score,roc_curve

In [18]:
modelos_metricas = []
curva_roc = []

In [26]:
def varios_modelos(X_train, X_test, y_train, y_test):

  classifiers = {"GaussianNB": GaussianNB(),
                "LogisiticRegression": LogisticRegression(),
                "KNeighborsClassifier": KNeighborsClassifier(),
                "DecisionTreeClassifier": DecisionTreeClassifier(),
                "RandomForestClassifier": RandomForestClassifier(),
                "XGBClassifier": XGBClassifier(),
                "AdaBoostClassifier": AdaBoostClassifier()}

  for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    y_pred= classifier.predict(X_test)
    bas= balanced_accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    f1s= f1_score(y_test, y_pred)
    avgp = average_precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    modelos_metricas.append([key, bas, roc_auc, f1s, avgp, accuracy])
    fpr, tpr, thresh = roc_curve(y_test, y_pred, pos_label=1)
    random_probs = [0 for i in range(len(y_test))]
    p_fpr, p_tpr, _ = roc_curve(y_test, random_probs, pos_label=1)
    curva_roc.append([key, fpr, tpr, thresh, p_fpr, p_tpr, _])

  tabla_modelos_metricas = pd.DataFrame(modelos_metricas , columns=['key', 'b_a_s', 'roc_auc_s', 'f1_s', 'avg_p_s', 'accuracy'])
  roc=pd.DataFrame(curva_roc, columns=['key', 'fpr', 'tpr', 'thresh', 'p_fpr', 'p_tpr', '_'])
  return tabla_modelos_metricas

In [27]:
varios_modelos(X_train, X_test, y_train, y_test)

,key,b_a_s,roc_auc_s,f1_s,avg_p_s,accuracy
0,GaussianNB,0.577048,0.577048,0.049425,0.005797,0.991914
1,LogisiticRegression,0.709852,0.709852,0.391262,0.154593,0.998292
2,KNeighborsClassifier,0.830111,0.830111,0.753608,0.579928,0.999437
3,DecisionTreeClassifier,0.926906,0.926906,0.868473,0.754655,0.999662
4,RandomForestClassifier,0.885416,0.885416,0.863115,0.756094,0.999681
5,XGBClassifier,0.931783,0.931783,0.915855,0.842051,0.999793
6,AdaBoostClassifier,0.733047,0.733047,0.625707,0.444188,0.999272
